In [1]:
from finn.util.basic import make_build_dir
from finn.util.visualization import showInNetron
    
build_dir = "/workspace/finn/0928"

In [3]:
import onnx
from finn.util.test import get_test_model_trained
import brevitas.onnx as bo
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.fold_constants import FoldConstants
from finn.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs

'''#kws = get_test_model_trained("KWS", 1, 1)
#bo.export_finn_onnx(kws, (1, 16, 16000, 1), build_dir + "/ckpt.t7.M5_11111.pth.finn.onnx")
model = ModelWrapper(build_dir + "/ckpt.t7.M5_11111_0902_2.pth.finn.onnx")
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(RemoveStaticGraphInputs())
model.save(build_dir + "/ckpt.t7.M5_11111.pth.finn_tidy.onnx")'''
model = ModelWrapper(build_dir + "/ckpt.t7.M5_11111.pth.tidy.onnx")

In [4]:
showInNetron(build_dir + "/ckpt.t7.M5_11111.pth.tidy.onnx")

Serving '/workspace/finn/0928/ckpt.t7.M5_11111.pth.tidy.onnx' at http://0.0.0.0:8081


In [5]:
from finn.util.pytorch import ToTensor
from finn.transformation.merge_onnx_models import MergeONNXModels
from finn.core.datatype import DataType

#model = ModelWrapper(build_dir + "/ckpt.t7.M5_11111_0920.pth.finn.onnx")
global_inp_name = model.graph.input[0].name
ishape = model.get_tensor_shape(global_inp_name)
# preprocessing: torchvision's ToTensor divides uint8 inputs by 255
totensor_pyt = ToTensor()
chkpt_preproc_name = build_dir+"/ckpt.t7.M5_11111.pth.finn_preproc.onnx"
bo.export_finn_onnx(totensor_pyt, ishape, chkpt_preproc_name)

# join preprocessing and core model
pre_model = ModelWrapper(chkpt_preproc_name)
model = model.transform(MergeONNXModels(pre_model))
# add input quantization annotation: UINT8 for all BNN-PYNQ models
global_inp_name = model.graph.input[0].name
model.set_tensor_datatype(global_inp_name, DataType.UINT8)
#model.set_tensor_datatype(global_inp_name, DataType.INT16)

/workspace/finn-base/src/finn/transformation/infer_data_layouts.py:114: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


In [6]:
from finn.transformation.insert_topk import InsertTopK
from finn.transformation.infer_datatypes import InferDataTypes
from finn.core.datatype import DataType


# postprocessing: insert Top-1 node at the end
model = model.transform(InsertTopK(k=1))
chkpt_name = build_dir+"/ckpt.t7.M5_11111.pth.finn_pre_post.onnx"
# tidy-up again
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())
model.save(chkpt_name)

showInNetron(build_dir+"/ckpt.t7.M5_11111.pth.finn_pre_post.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/finn/0928/ckpt.t7.M5_11111.pth.finn_pre_post.onnx' at http://0.0.0.0:8081


In [7]:
from finn.util.visualization import showSrc
from finn.transformation.streamline import Streamline
showSrc(Streamline)

class Streamline(Transformation):
    """Apply the streamlining transform, see arXiv:1709.04060."""

    def apply(self, model):
        streamline_transformations = [
            ConvertSubToAdd(),
            ConvertDivToMul(),
            BatchNormToAffine(),
            ConvertSignToThres(),
            MoveMulPastMaxPool(),
            MoveScalarLinearPastInvariants(),
            AbsorbSignBiasIntoMultiThreshold(),
            MoveAddPastMul(),
            MoveScalarAddPastMatMul(),
            MoveAddPastConv(),
            MoveScalarMulPastMatMul(),
            MoveScalarMulPastConv(),
            MoveAddPastMul(),
            CollapseRepeatedAdd(),
            CollapseRepeatedMul(),
            MoveMulPastMaxPool(),
            AbsorbAddIntoMultiThreshold(),
            FactorOutMulSignMagnitude(),
            AbsorbMulIntoMultiThreshold(),
            Absorb1BitMulIntoMatMul(),
            Absorb1BitMulIntoConv(),
            RoundAndClipThresholds(),
        ]
        for tr

In [8]:
from finn.transformation.streamline.reorder import MoveScalarLinearPastInvariants
import finn.transformation.streamline.absorb as absorb

model = ModelWrapper(build_dir+"/ckpt.t7.M5_11111.pth.finn_pre_post.onnx")
# move initial Mul (from preproc) past the Reshape
model = model.transform(MoveScalarLinearPastInvariants())
chkpt_name = build_dir+"/ckpt.t7.M5_11111.pth.finn_pre_stream.onnx"
model.save(chkpt_name)
#no use!!the picture is the same as last one 
showInNetron(build_dir+"/ckpt.t7.M5_11111.pth.finn_pre_stream.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/finn/0928/ckpt.t7.M5_11111.pth.finn_pre_stream.onnx' at http://0.0.0.0:8081


In [9]:
# streamline
#model = model.transform(Streamline())
model = model.transform(Streamline())
model.save(build_dir+"/ckpt.t7.M5_11111.pth.finn_streamlined.onnx")
showInNetron(build_dir+"/ckpt.t7.M5_11111.pth.finn_streamlined.onnx")

/workspace/finn/src/finn/transformation/streamline/__init__.py:102: UserWarning: <class 'finn.transformation.general.ConvertSubToAdd'>
  warnings.warn(str(type(trn)))
/workspace/finn/src/finn/transformation/streamline/__init__.py:102: UserWarning: <class 'finn.transformation.general.ConvertDivToMul'>
  warnings.warn(str(type(trn)))
/workspace/finn/src/finn/transformation/streamline/__init__.py:102: UserWarning: <class 'finn.transformation.batchnorm_to_affine.BatchNormToAffine'>
  warnings.warn(str(type(trn)))
/workspace/finn/src/finn/transformation/streamline/__init__.py:102: UserWarning: <class 'finn.transformation.streamline.sign_to_thres.ConvertSignToThres'>
  warnings.warn(str(type(trn)))
/workspace/finn/src/finn/transformation/streamline/__init__.py:102: UserWarning: <class 'finn.transformation.streamline.reorder.MoveMulPastMaxPool'>
  warnings.warn(str(type(trn)))
/workspace/finn/src/finn/transformation/streamline/__init__.py:102: UserWarning: <class 'finn.transformation.streamli

Stopping http://0.0.0.0:8081
Serving '/workspace/finn/0928/ckpt.t7.M5_11111.pth.finn_streamlined.onnx' at http://0.0.0.0:8081


In [10]:
from finn.transformation.streamline import Streamline
from finn.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from finn.transformation.bipolar_to_xnor import ConvertBipolarMatMulToXnorPopcount
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline.reorder import MakeMaxPoolNHWC, MoveScalarLinearPastInvariants
from finn.transformation.infer_data_layouts import InferDataLayouts
from finn.transformation.general import RemoveUnusedTensors


model = ModelWrapper(build_dir+"/ckpt.t7.M5_11111.pth.finn_streamlined.onnx")
model = model.transform(LowerConvsToMatMul())
model = model.transform(MakeMaxPoolNHWC())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(ConvertBipolarMatMulToXnorPopcount())
model.save(build_dir+"/ckpt.t7.M5_11111.pth.finn_before_second_streamlined.onnx")

In [11]:
showInNetron(build_dir+"/ckpt.t7.M5_11111.pth.finn_before_second_streamlined.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/finn/0928/ckpt.t7.M5_11111.pth.finn_before_second_streamlined.onnx' at http://0.0.0.0:8081


In [12]:
model = model.transform(Streamline())
model.save(build_dir+"/ckpt.t7.M5_11111.pth.finn_after_second_streamlined.onnx")

In [13]:
showInNetron(build_dir+"/ckpt.t7.M5_11111.pth.finn_after_second_streamlined.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/finn/0928/ckpt.t7.M5_11111.pth.finn_after_second_streamlined.onnx' at http://0.0.0.0:8081


In [14]:
model = model.transform(absorb.AbsorbScalarMulAddIntoTopK())
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())


model.save(build_dir+"/ckpt.t7.M5_11111.pth.finn_ready_for_hls_conversion.onnx")

In [15]:
showInNetron(build_dir+"/ckpt.t7.M5_11111.pth.finn_ready_for_hls_conversion.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/finn/0928/ckpt.t7.M5_11111.pth.finn_ready_for_hls_conversion.onnx' at http://0.0.0.0:8081


In [16]:
import finn.transformation.fpgadataflow.convert_to_hls_layers as to_hls
from finn.transformation.fpgadataflow.create_dataflow_partition import (
    CreateDataflowPartition,
)
"""
from finn.transformation.move_reshape import RemoveCNVtoFCFlatten
from finn.custom_op.registry import getCustomOp
from finn.transformation.infer_data_layouts import InferDataLayouts
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild
from finn.core.modelwrapper import ModelWrapper
from finn.custom_op.registry import getCustomOp
from finn.util.basic import make_build_dir
from finn.util.visualization import showInNetron
import onnx
from finn.util.test import get_test_model_trained
import brevitas.onnx as bo
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.fold_constants import FoldConstants
from finn.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from finn.transformation.streamline import Streamline
from finn.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from finn.transformation.bipolar_to_xnor import ConvertBipolarMatMulToXnorPopcount
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline.reorder import MakeMaxPoolNHWC, MoveScalarLinearPastInvariants
from finn.transformation.infer_data_layouts import InferDataLayouts
from finn.transformation.general import RemoveUnusedTensors
build_dir = "/workspace/finn"
"""
# choose the memory mode for the MVTU units, decoupled or const
mem_mode = "decoupled"

model = ModelWrapper(build_dir + "/ckpt.t7.M5_11111.pth.finn_ready_for_hls_conversion.onnx")
model = model.transform(to_hls.InferBinaryStreamingFCLayer(mem_mode))
model = model.transform(to_hls.InferQuantizedStreamingFCLayer(mem_mode))
# TopK to LabelSelect
model = model.transform(to_hls.InferLabelSelectLayer())


In [17]:
# input quantization (if any) to standalone thresholding
model = model.transform(to_hls.InferThresholdingLayer())
model = model.transform(to_hls.InferConvInpGen())
model.save(build_dir+"/ckpt.t7.M5_11111.pth.finn_test.onnx")
showInNetron(build_dir+"/ckpt.t7.M5_11111.pth.finn_test.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/finn/0928/ckpt.t7.M5_11111.pth.finn_test.onnx' at http://0.0.0.0:8081


In [18]:
from finn.transformation.move_reshape import RemoveCNVtoFCFlatten

model = ModelWrapper(build_dir + "/ckpt.t7.M5_11111.pth.finn_test.onnx")
model = model.transform(to_hls.InferStreamingMaxPool())
# get rid of Reshape(-1, 1) operation between hlslib nodes
model.save(build_dir + "/ckpt.t7.M5_11111.pth.BRemoveCNVtoFCFlatten.onnx")
model = model.transform(RemoveCNVtoFCFlatten())
model.save(build_dir + "/ckpt.t7.M5_11111.pth.ARemoveCNVtoFCFlatten.onnx")
# get rid of Tranpose -> Tranpose identity seq
model = model.transform(absorb.AbsorbConsecutiveTransposes())

In [19]:
showInNetron(build_dir + "/ckpt.t7.M5_11111.pth.BRemoveCNVtoFCFlatten.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/finn/0928/ckpt.t7.M5_11111.pth.BRemoveCNVtoFCFlatten.onnx' at http://0.0.0.0:8081


In [20]:
from finn.custom_op.registry import getCustomOp
# infer tensor data layouts
model = model.transform(InferDataLayouts())
parent_model = model.transform(CreateDataflowPartition())
parent_model.save(build_dir + "/ckpt.t7.M5_11111.pth.finn_dataflow_parent.onnx")
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
sdp_node = getCustomOp(sdp_node)
dataflow_model_filename = sdp_node.get_nodeattr("model")
# save the dataflow partition with a different name for easier access
dataflow_model = ModelWrapper(dataflow_model_filename)
dataflow_model.save(build_dir + "/ckpt.t7.M5_11111.pth.finn_dataflow_model.onnx")

In [21]:
showInNetron(build_dir + "/ckpt.t7.M5_11111.pth.finn_dataflow_model.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/finn/0928/ckpt.t7.M5_11111.pth.finn_dataflow_model.onnx' at http://0.0.0.0:8081


In [22]:
showInNetron(build_dir + "/ckpt.t7.M5_11111.pth.finn_dataflow_parent.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/finn/0928/ckpt.t7.M5_11111.pth.finn_dataflow_parent.onnx' at http://0.0.0.0:8081


In [23]:
#showInNetron("/tmp/finn_dev_lab2312/dataflow_partition0_ewnvlgpe/df_model.onnx")

In [24]:
'''from finn.custom_op.registry import getCustomOp
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
sdp_node = getCustomOp(sdp_node)
dataflow_model_filename = sdp_node.get_nodeattr("model")
showInNetron(dataflow_model_filename)'''

'from finn.custom_op.registry import getCustomOp\nsdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]\nsdp_node = getCustomOp(sdp_node)\ndataflow_model_filename = sdp_node.get_nodeattr("model")\nshowInNetron(dataflow_model_filename)'

In [25]:
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild
from finn.core.modelwrapper import ModelWrapper
from finn.custom_op.registry import getCustomOp
from finn.util.basic import make_build_dir
from finn.util.visualization import showInNetron
import onnx
from finn.util.test import get_test_model_trained
import brevitas.onnx as bo
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.fold_constants import FoldConstants
from finn.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs

build_dir = "/workspace/finn"
model = ModelWrapper(build_dir + "/ckpt.t7.M5_11111.pth.finn_dataflow_model.onnx")
fc_layers = model.get_nodes_by_op_type("StreamingFCLayer_Batch")

In [26]:
showInNetron(build_dir + "/ckpt.t7.M5_11111.pth.finn_dataflow_parent.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/finn/ckpt.t7.M5_11111.pth.finn_dataflow_parent.onnx' at http://0.0.0.0:8081


In [27]:
showInNetron(build_dir + "/ckpt.t7.M5_11111.pth.finn_dataflow_model.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/finn/ckpt.t7.M5_11111.pth.finn_dataflow_model.onnx' at http://0.0.0.0:8081


In [28]:
# each tuple is (PE, SIMD, in_fifo_depth, ramstyle) for a layer
'''
folding = [
    (1, 1, 128, "auto"),
    (1, 1, 128, "auto"),
    (1, 1, 128, "auto"),
    (1, 1, 128, "auto"),
    (1, 1, 128, "auto"),
    (1, 1, 128, "auto"),
    (1, 1, 128, "auto"),
    (1, 1, 128, "auto"),
    (1, 1, 128, "auto"),
    (1, 1, 3, "auto"),
]
'''
folding = [
    (8, 1, 128, "distributed"),
    (8, 4, 32, "distributed"),
    (8, 4, 32, "auto"),
    (8, 4, 32, "auto"),
    (8, 4, 32, "block"),
    (8, 4, 32, "auto"),
    (8, 4, 32, "auto"),
    (8, 4, 128, "auto"),
    (8, 4, 128, "auto"),
    (1, 8, 3, "block"),
]

In [29]:
URAM_counter = 0

In [30]:
'''for fcl, (pe, simd, ififodepth, ramstyle) in zip(fc_layers, folding):
    fcl_inst = getCustomOp(fcl)
    
    if URAM_counter < 5:
        #fcl_inst.set_nodeattr("runtime_writeable_weights", 1)
        fcl_inst.set_nodeattr("mem_mode", "external")
    else:
        fcl_inst.set_nodeattr("mem_mode", "decoupled")
        fcl_inst.set_nodeattr("ram_style", ramstyle)
    URAM_counter +=1
    fcl_inst.set_nodeattr("PE", pe)
    fcl_inst.set_nodeattr("SIMD", simd)
    fcl_inst.set_nodeattr("inFIFODepth", ififodepth)'''
    #fcl_inst.set_nodeattr("mem_mode", "external")
    #fcl_inst.set_nodeattr("ram_style", ramstyle)

for fcl, (pe, simd, ififodepth, ramstyle) in zip(fc_layers, folding):
    fcl_inst = getCustomOp(fcl)
    '''if URAM_counter < 7:
        fcl_inst.set_nodeattr("runtime_writeable_weights", 1)
        print(URAM_counter)
    URAM_counter +=1'''
    
    fcl_inst.set_nodeattr("PE", pe)
    fcl_inst.set_nodeattr("SIMD", simd)
    fcl_inst.set_nodeattr("inFIFODepth", ififodepth) 
    fcl_inst.set_nodeattr("ram_style", ramstyle)
    

In [31]:
URAM_counter = 0
# use same SIMD values for the sliding window operators
#swg_layers = model.get_nodes_by_op_type("ConvolutionInputGenerator")
swg_layers = model.get_nodes_by_op_type("ConvolutionInputGenerator1D")
for i in range(len(swg_layers)):
    swg_inst = getCustomOp(swg_layers[i])
    print(len(swg_layers))
    simd = folding[i][1]
    if URAM_counter < 1:
        ram_style = "ultra"
    else :
        ram_style = "distributed"
    print(URAM_counter)
    URAM_counter +=1
    
    swg_inst.set_nodeattr("SIMD", simd)
    swg_inst.set_nodeattr("ram_style", ram_style)

model = model.transform(GiveUniqueNodeNames())
model.save(build_dir + "/ckpt.t7.M5_11111.pth.finn_folded.onnx")

9
0
9
1
9
2
9
3
9
4
9
5
9
6
9
7
9
8


In [32]:
showInNetron(build_dir + "/ckpt.t7.M5_11111.pth.finn_folded.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/finn/ckpt.t7.M5_11111.pth.finn_folded.onnx' at http://0.0.0.0:8081


In [33]:
# print the names of the supported PYNQ boards
from finn.util.basic import pynq_part_map
print(pynq_part_map.keys())

dict_keys(['Ultra96', 'Pynq-Z1', 'Pynq-Z2', 'ZCU102', 'ZCU104'])


In [34]:
# change this if you have a different PYNQ board, see list above
pynq_board = "ZCU104"
fpga_part = pynq_part_map[pynq_board]
target_clk_ns = 10000

In [35]:
model = ModelWrapper(build_dir+"/ckpt.t7.M5_11111.pth.finn_folded.onnx")
model = model.transform(ZynqBuild(platform = pynq_board, period_ns = target_clk_ns))

/workspace/finn/src/finn/custom_op/fpgadataflow/convolutioninputgenerator1d.py:105: UserWarning: get_folded_input_shape: 1, 1
  warnings.warn("get_folded_input_shape: %s, %s" %(str(ifm_ch), str(simd)))
/workspace/finn/src/finn/custom_op/fpgadataflow/convolutioninputgenerator1d.py:133: UserWarning: get_folded_output_shape: 1, 1
  warnings.warn("get_folded_output_shape: %s, %s" %(str(ifm_ch), str(simd)))
/workspace/finn/src/finn/custom_op/fpgadataflow/convolutioninputgenerator1d.py:105: UserWarning: get_folded_input_shape: 48, 4
  warnings.warn("get_folded_input_shape: %s, %s" %(str(ifm_ch), str(simd)))
/workspace/finn/src/finn/custom_op/fpgadataflow/convolutioninputgenerator1d.py:133: UserWarning: get_folded_output_shape: 48, 4
  warnings.warn("get_folded_output_shape: %s, %s" %(str(ifm_ch), str(simd)))
/workspace/finn/src/finn/custom_op/fpgadataflow/convolutioninputgenerator1d.py:105: UserWarning: get_folded_input_shape: 96, 4
  warnings.warn("get_folded_input_shape: %s, %s" %(str(ifm_

In [36]:
model.save(build_dir + "/ckpt.t7.M5_11111.pth.finn_synth.onnx")

In [37]:
showInNetron(build_dir + "/ckpt.t7.M5_11111.pth.finn_synth.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/finn/ckpt.t7.M5_11111.pth.finn_synth.onnx' at http://0.0.0.0:8081


In [38]:
import os

# set up the following values according to your own environment
# FINN will use ssh to deploy and run the generated accelerator
ip = os.getenv("PYNQ_IP", "192.168.0.3")
username = os.getenv("PYNQ_USERNAME", "xilinx")
password = os.getenv("PYNQ_PASSWORD", "xilinx")
port = os.getenv("PYNQ_PORT", 22)
target_dir = os.getenv("PYNQ_TARGET_DIR", "/home/xilinx/finn_dev_lab2312")
# set up ssh options to only allow publickey authentication
options = "-o PreferredAuthentications=publickey -o PasswordAuthentication=no"

# test access to PYNQ board
! ssh {options} {username}@{ip} -p {port} cat /var/run/motd.dynamic

Welcome to PYNQ Linux, based on Ubuntu 18.04 (GNU/Linux 5.4.0-xilinx-v2020.1 aarch64)



In [39]:
from finn.transformation.fpgadataflow.make_deployment import DeployToPYNQ

model = model.transform(DeployToPYNQ("192.168.0.3", "22", "xilinx", "xilinx", "/home/xilinx/finn_dev_lab2312"))
model.save(build_dir + "/ckpt.t7.M5_11111.pth.pynq_deploy.onnx")

In [40]:
target_dir_pynq = target_dir + "/" + model.get_metadata_prop("pynq_deployment_dir").split("/")[-1]
target_dir_pynq

'/home/xilinx/finn_dev_lab2312/pynq_deployment_nqrj6c5k'

In [ ]:
model = ModelWrapper(build_dir + "/ckpt.t7.M5_11111.pth.pynq_deploy.onnx")
sdp_node_middle = getCustomOp(model.graph.node[1])
postsynth_layers = sdp_node_middle.get_nodeattr("model")

#showInNetron(postsynth_layers)

In [ ]:
model = ModelWrapper(postsynth_layers)
model.model.metadata_props

In [ ]:
model = ModelWrapper(build_dir + "/ckpt.t7.M5_11111.pth.pynq_deploy.onnx")
model.model.metadata_props

In [ ]:
! ls {model.get_metadata_prop("vivado_pynq_proj")}

In [ ]:
model.model.metadata_props

In [ ]:
target_dir_pynq = '/home/xilinx/finn_dev_lab2312/pynq_deployment_kkl5g6nh'

In [ ]:
! ssh -o PreferredAuthentications=publickey -o PasswordAuthentication=no xilinx@192.168.0.3 -p 22 'ls -l /home/xilinx/finn_dev_lab2312/pynq_deployment_kkl5g6nh'

In [ ]:
import pkg_resources as pk
import matplotlib.pyplot as plt
import numpy as np

fn = pk.resource_filename("finn.qnn-data", "KWS/up2.npy")
x = np.load(fn)
x = x.reshape(1, 16000, 1)
plt.imshow(x)

In [ ]:
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild
from finn.core.modelwrapper import ModelWrapper
from finn.custom_op.registry import getCustomOp
from finn.util.basic import make_build_dir
from finn.util.visualization import showInNetron
import onnx
from finn.util.test import get_test_model_trained
import brevitas.onnx as bo
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.fold_constants import FoldConstants
from finn.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from finn.util.visualization import showInNetron

import numpy as np
from finn.core.onnx_exec import execute_onnx


build_dir = "/workspace/finn"
model = ModelWrapper(build_dir + "/ckpt.t7.M5_11111.pth.pynq_deployonboard.onnx")
iname = model.graph.input[0].name
oname = model.graph.output[0].name
ishape = model.get_tensor_shape(iname)
print(iname)
print("Expected network input shape is " + str(ishape))

In [ ]:
showInNetron(build_dir + "/ckpt.t7.M5_11111.pth.pynq_deployonboard.onnx")

In [ ]:
input_dict = {iname: x.reshape(ishape)}
input_dict

In [ ]:
ret = execute_onnx(model, input_dict, False)

In [ ]:
ret[oname]

In [ ]:
! ssh -o PreferredAuthentications=publickey -o PasswordAuthentication=no xilinx@192.168.0.3 -p 22 'echo xilinx | sudo -S pip3 install git+https://github.com/fbcotter/dataset_loading.git@0.0.4#egg=dataset_loading'

In [ ]:
from finn.core.throughput_test import throughput_test_remote

model = ModelWrapper(build_dir + "/ckpt.t7.M5_11111.pth.pynq_deployonboard.onnx")
res = throughput_test_remote(model, 10000)
print("Network metrics:")
for key in res:
    print(str(key) + ": " + str(res[key]))